In [1]:
import pandas as pd
import requests
import zipfile
import io
import os
from shapely.geometry import Point, LineString
from shapely.ops import transform
import pyproj

# Configuration
GTFS_URL = "https://rrgtfsfeeds.s3.amazonaws.com/gtfs_subway.zip"
DATA_DIR = "data/static"
os.makedirs(DATA_DIR, exist_ok=True)

## 1. Download and Extract GTFS Data

In [2]:
print(f"Downloading GTFS data from {GTFS_URL}...")
r = requests.get(GTFS_URL)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(DATA_DIR)
print("Download and extraction complete.")

Download and extraction complete.


## 2. Load Shapes and Stops

In [3]:
# Load necessary files
shapes_df = pd.read_csv(os.path.join(DATA_DIR, "shapes.txt"))
stops_df = pd.read_csv(os.path.join(DATA_DIR, "stops.txt"))
trips_df = pd.read_csv(os.path.join(DATA_DIR, "trips.txt"))
routes_df = pd.read_csv(os.path.join(DATA_DIR, "routes.txt"))

print("Shapes:", shapes_df.shape)
print("Stops:", stops_df.shape)
print("Trips:", trips_df.shape)

Shapes: (149834, 4)
Stops: (1488, 6)
Trips: (20304, 6)
